In [54]:
import json
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [55]:
encoder = SentenceTransformer("all-mpnet-base-v2")

In [56]:
with open("schema_docs.json", "r") as f:
    schema_docs = json.load(f)

In [57]:
vectors = encoder.encode(schema_docs).tolist()

In [58]:
client = QdrantClient(':memory:')

client.recreate_collection(
    collection_name = 'schema',
    vectors_config = models.VectorParams(size = len(vectors[0]), distance = models.Distance.COSINE)
)

client.upsert(
    collection_name = 'schema',
    points = [
        models.PointStruct(id = i, vector = vectors[i], payload = {'text': schema_docs[i]}) for i in range(len(schema_docs))
    ]
)

/var/folders/ph/8_vyygs16cx718tn3nsdccww0000gn/T/ipykernel_27692/1050872298.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [62]:
query = 'Average shipping delay per shipping mode.'
query_vec = encoder.encode(query).tolist()

results = client.search(
    collection_name = 'schema',
    query_vector = query_vec,
    limit = 5
)

for r in results:
    print(f"Score: {r.score:.4f}, Text: {r.payload['text']}")

Score: 0.5300, Text: Column ship_mode: The shipping method selected for the order (e.g., Standard Class, Second Class, First Class, Same Day). Important for analyzing the relationship between shipping type, delivery time, and customer satisfaction or cost.
Score: 0.3531, Text: Column ship_date: The calendar date when the order was shipped to the customer. Useful for analyzing delivery times, fulfillment efficiency, and operational logistics.
Score: 0.2062, Text: Column sales: The total sales revenue for this order line (in U.S. dollars). This is a key financial metric for analyzing revenue performance, growth trends, and product contribution. Often aggregated (SUM) for revenue reports.
Score: 0.1996, Text: Column region: The broad U.S. geographic region (West, East, Central, South) where the order was placed. Often used for high-level geographic revenue comparisons and regional performance trends.
Score: 0.1989, Text: Table superstore: This table contains transaction-level retail sales

/var/folders/ph/8_vyygs16cx718tn3nsdccww0000gn/T/ipykernel_27692/1285302100.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
